# Rekurencyjne Sieci Neuronowe (RNN)

Materiały w większości pochodzą od pracowników wydziału matematyki i informatyki.

### Importy i Utilsy  (odpalić i schować)

In [827]:
# imports 
import torch
import os
import unicodedata
import string
import numpy as np
from typing import Tuple, Optional, List

from torch.nn.functional import cross_entropy

import matplotlib.pyplot as plt 
from sklearn.metrics import f1_score

from torch.utils.data import Dataset, DataLoader

all_letters = string.ascii_letters
n_letters = len(all_letters)


class ListDataset(Dataset):
    
    def __init__(self, data, targets):
        
        self.data = data
        self.targets = targets
        
    def __getitem__(self, ind):
        
        return self.data[ind], self.targets[ind]
    
    def __len__(self):
        return len(self.targets)

    
def unicode_to__ascii(s: str) -> str:
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn'
                                                                 and c in all_letters)
                   

def read_lines(filename: str) -> List[str]:
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    return [unicode_to__ascii(line) for line in lines]


def letter_to_index(letter: str) -> int:
    return all_letters.find(letter)


def line_to_tensor(line: str) -> torch.Tensor:
    tensor = torch.zeros(len(line), n_letters)
    for i, letter in enumerate(line):
        tensor[i][letter_to_index(letter)] = 1
    return tensor

## Dane sekwencyjne

Modele, którymi zajmowaliśmy się wcześniej zakładały konkretny kształt danych. Dla przykładu klasyczna sieć neuronowa fully-connected dla MNISTa zakładała, że na wejściu dostanie wektory rozmiaru 784 - dla wektorów o innej wymiarowości i innych obiektów model zwyczajnie nie będzie działać.

Takie założenie bywa szczególnie niewygodne przy pracy z niektórymi typami danych, takimi jak:
* językiem naturalny (słowa czy zdania mają zadanej z góry liczby znaków)
* szeregi czasowe (dane giełdowe ciągną się właściwie w nieskończoność) 
* dźwięk (nagrania mogą być krótsze lub dłuższe).

Do rozwiązania tego problemu służą rekuencyjne sieci neuronowe (*recurrent neural networks, RNNs*), które zapamiętują swój stan z poprzedniej iteracji.

### Ładowanie danych
Poniższe dwie komórki ściągają dataset nazwisk z 18 różnych narodowości. Każda litera w danym nazwisku jest zamieniana na jej indeks z alfabetu w postaci kodowania "one-hot". Inaczej mówiąc, każde nazwisko jest binarną macierzą rozmiaru `n_letters` $\times$ `len(name)`. 

Dodatkowo, ponieważ ten dataset jest mocno niezbalansowany, użyjemy specjalnego samplera do losowania przykładów treningowych, tak aby do uczenia sieć widziała tyle samo przykładów z każdej klasy.

Ponieważ nazwiska mogą mieć różne długości będziemy rozważać `batch_size = 1` w tym notebooku (choć implementacje modeli powinny działać dla dowolnych wartości `batch_size`!)

In [2]:

!wget https://download.pytorch.org/tutorial/data.zip
!unzip data.zip


--2022-12-12 16:09:50--  https://download.pytorch.org/tutorial/data.zip
Resolving download.pytorch.org (download.pytorch.org)... 143.204.215.66, 143.204.215.83, 143.204.215.17, ...
Connecting to download.pytorch.org (download.pytorch.org)|143.204.215.66|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2882130 (2.7M) [application/zip]
Saving to: ‘data.zip’

data.zip            100%[===================>]   2.75M  14.8MB/s    in 0.2s    

2022-12-12 16:09:51 (14.8 MB/s) - ‘data.zip’ saved [2882130/2882130]

Archive:  data.zip
   creating: data/
  inflating: data/eng-fra.txt        
   creating: data/names/
  inflating: data/names/Arabic.txt   
  inflating: data/names/Chinese.txt  
  inflating: data/names/Czech.txt    
  inflating: data/names/Dutch.txt    
  inflating: data/names/English.txt  
  inflating: data/names/French.txt   
  inflating: data/names/German.txt   
  inflating: data/names/Greek.txt    
  inflating: data/names/Irish.txt    
  inflating: data/name

In [5]:
# NOTE: you can change the seed or remove it completely if you like
torch.manual_seed(1337)

data_dir = 'data/names'

# Build the category_lines dictionary, a list of names per language
category_lines = {}
all_categories = []

data = []
targets = [] 
label_to_idx = {}

# read each natonality file and process data 
for label, file_name in enumerate(os.listdir(data_dir)):
    
    label_to_idx[label] = file_name.split('.')[0].lower()
    
    names = read_lines(os.path.join(data_dir, file_name))
    data += [line_to_tensor(name) for name in names]
    targets += len(names) * [label]

# split into train and test indices
test_frac = 0.1
n_test = int(test_frac * len(targets))
test_ind = np.random.choice(len(targets), size=n_test, replace=False)
train_ind = np.setdiff1d(np.arange(len(targets)), test_ind)

targets = torch.tensor(targets)
train_targets = targets[train_ind]

# calculate weights for BalancedSampler
uni, counts = np.unique(train_targets, return_counts=True)
weight_per_class = len(targets) / counts
weight = [weight_per_class[c] for c in train_targets]
# preapre the sampler
sampler = torch.utils.data.sampler.WeightedRandomSampler(weights=weight, num_samples=len(weight)) 

train_dataset = ListDataset(data=[x for i, x in enumerate(data) if i in train_ind], targets=train_targets)
train_loader = DataLoader(train_dataset, shuffle=False, batch_size=1, sampler=sampler)

test_dataset = ListDataset(data=[x for i, x in enumerate(data) if i in test_ind], targets=targets[test_ind])
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=1)

In [6]:
# check out the content of the dataset
for i, (x, y) in enumerate(train_loader):
    break

print("x.shape:", x.shape)
print("name: ", end="")
for letter_onehot in x[0]:
    print(all_letters[torch.argmax(letter_onehot)], end="")

print("\ny:", label_to_idx[y.item()])

x.shape: torch.Size([1, 6, 52])
name: OKelly
y: irish


<h4> Zadanie 1. </h4>

Zaimplementuj "zwykłą" sieć rekurencyjną. 
![rnn](https://colah.github.io/posts/2015-08-Understanding-LSTMs/img/RNN-unrolled.png)

Przyjmijmy, że stan ukryty ma wymiarowość $H$. Wtedy komórka powinna być warstwą liniową o postaci: $\tanh(W^T [x_t, h_{t-1}] + b)$, gdzie $x_t \in \mathbb{R}^{D}$ to wejście w kroku $t$, $h_{t-1} \in \mathbb{R}^{H}$ to stan ukryty z poprzedniego kroku a $W \in \mathbb{R}^{(H + D) \times H}$ i $b \in \mathbb{R}^H$ to parametry naszego modelu.

* W klasie `RNN` należy zainicjalizować potrzebne wagi oraz zaimplementować główną logikę dla pojedynczej chwili czasowej $x_t$
* Wyjście z sieci może mieć dowolny rozmiar, potrzebna jest również warstwa przekształacjąca $H$-wymiarowy stan ukryty na wyjście (o takiej wymiarowości ile mamy klas w naszym problemie). 
* W pętli uczenia należy dodać odpowiednie wywołanie sieci. HINT: pamiętać o iterowaniu po wymiarze "czasowym".


In [829]:
class RNN(torch.nn.Module):
    
    def __init__(self, 
                 input_size: int,
                 hidden_size: int, 
                 output_size: int):
        """
        :param input_size: int
            Dimensionality of the input vector
        :param hidden_size: int
            Dimensionality of the hidden space
        :param output_size: int
            Desired dimensionality of the output vector
        """
        super(RNN, self).__init__()

        self.hidden_size = hidden_size

        self.input_to_hidden = nn.Linear(input_size + hidden_size, hidden_size)
        
        self.hidden_to_output = nn.Linear(input_size + hidden_size, output_size)
    
    # for the sake of simplicity a single forward will process only a single timestamp 
    def forward(self, 
                input: torch.tensor, 
                hidden: torch.tensor) -> Tuple[torch.tensor, torch.tensor]:
        """
        :param input: torch.tensor 
            Input tesnor for a single observation at timestep t
            shape [batch_size, input_size]
        :param hidden: torch.tensor
            Representation of the memory of the RNN from previous timestep
            shape [batch_size, hidden_size]
        """
        combined = torch.cat([input, hidden], 1) 
        hidden = self.input_to_hidden(combined)
        output =  self.hidden_to_output(combined)
        return output, hidden
    
    def init_hidden(self, batch_size: int) -> torch.Tensor:
        """
        Returns initial value for the hidden state
        """
        return torch.zeros(batch_size, self.hidden_size, requires_grad=True)

### Pętla uczenia

In [833]:
n_class = len(label_to_idx)

# initialize network and optimizer
rnn = RNN(n_letters, 256, n_class)
optimizer = torch.optim.SGD(rnn.parameters(), lr=0.01)   

# we will train for only a single epoch 
epochs = 1

criterion = nn.NLLLoss()
# main loop
for epoch in range(epochs):
    
    loss_buffer = []
    
    for i, (x, y) in enumerate(train_loader):  
        
        
        optimizer.zero_grad()
        # get initial hidden state
        hidden = rnn.init_hidden(x.shape[0])
        
        # get output for the sample, remember that we treat it as a sequence
        # so you need to iterate over the 2nd, time dimensiotn
        seq_len = x.shape[1]
        for m in range(seq_len):
                output, hidden = rnn(x[:,m], hidden) #najlepiej w pętli; uwaga RNN przy okazji generuje hidden
        loss = cross_entropy(output, y)
        loss.backward()
        optimizer.step()  
        
        loss_buffer.append(loss.item())
        
        if i % 1000 == 1:
            print(f"Epoch: {epoch} Progress: {100 * i/len(train_loader):2.0f}% Loss: {np.mean(loss_buffer):.3f}")
            loss_buffer = []
    

# evaluate on the test set
with torch.no_grad():
    ps = []
    ys = []
    correct = 0
    for i, (x, y) in enumerate(test_loader):
        ys.append(y.numpy())

        hidden = rnn.init_hidden(x.shape[0])
        seq_len = x.shape[1]
        for m in range(seq_len):
                output, hidden = rnn(x[:,m], hidden)
        

        pred = output.argmax(dim=1)
        ps.append(pred.cpu().numpy())
    
    ps = np.concatenate(ps, axis=0)
    ys = np.concatenate(ys, axis=0)
    f1 = f1_score(ys, ps, average='weighted')
    
    print(f"Final F1 score: {f1:.2f}")
    assert f1 > 0.15, "You should get over 0.15 f1 score, try changing some hyperparams!"

Epoch: 0 Progress:  0% Loss: 2.815
Epoch: 0 Progress:  6% Loss: 2.846
Epoch: 0 Progress: 11% Loss: 2.697
Epoch: 0 Progress: 17% Loss: 2.474
Epoch: 0 Progress: 22% Loss: 2.328
Epoch: 0 Progress: 28% Loss: 2.240
Epoch: 0 Progress: 33% Loss: 2.077
Epoch: 0 Progress: 39% Loss: 2.056
Epoch: 0 Progress: 44% Loss: 2.000
Epoch: 0 Progress: 50% Loss: 1.948
Epoch: 0 Progress: 55% Loss: 1.891
Epoch: 0 Progress: 61% Loss: 1.821
Epoch: 0 Progress: 66% Loss: 1.842
Epoch: 0 Progress: 72% Loss: 1.829
Epoch: 0 Progress: 77% Loss: 1.768
Epoch: 0 Progress: 83% Loss: 1.736
Epoch: 0 Progress: 89% Loss: 1.773
Epoch: 0 Progress: 94% Loss: 1.655
Epoch: 0 Progress: 100% Loss: 1.904
Final F1 score: 0.22


<h4> Zadanie 2. </h4> 

Zaimplementuj funkcje `predict`, która przyjmuje nazwisko w postaci stringa oraz model RNN i wypisuje 3 najlepsze predykcje narodowości dla tego nazwiska razem z ich logitami.

**Hint**: Przyda się tutaj jedna z funkcji z pierwszej komórki notebooka.

In [ ]:
def decode_nationality(nationality_idx):
    nationalities = all_categories
    return nationalities[nationality_idx]

In [834]:
def predict(name: str, rnn: RNN):
    name = line_to_tensor(name)
    hidden = rnn.init_hidden(1)
    for i in range(len(name)):
        output, hidden = rnn(name[i].unsqueeze(0), hidden)
    top_3_preds = output.topk(3)

    for i in range(3):
        nationality = decode_nationality(top_3_preds[1][0][i].item())
        print(f"\tPrediction {i+1}: {nationality[1]} (Score: {top_3_preds[0][0][i].item():.2f})")

In [835]:
some_names = ["Satoshi", "Jackson", "Schmidhuber", "Hinton", "Kowalski"]

for name in some_names:
    print(name)
    predict(name, rnn)

Satoshi
	Prediction 1: Japanese (Score: 3.04)
	Prediction 2: Italian (Score: 2.44)
	Prediction 3: Polish (Score: 1.78)
Jackson
	Prediction 1: Scottish (Score: 4.43)
	Prediction 2: English (Score: 2.27)
	Prediction 3: French (Score: 1.74)
Schmidhuber
	Prediction 1: Russian (Score: 1.67)
	Prediction 2: German (Score: 1.65)
	Prediction 3: Polish (Score: 1.55)
Hinton
	Prediction 1: Scottish (Score: 2.47)
	Prediction 2: English (Score: 2.42)
	Prediction 3: French (Score: 1.50)
Kowalski
	Prediction 1: Polish (Score: 4.39)
	Prediction 2: Czech (Score: 2.30)
	Prediction 3: Japanese (Score: 2.27)


<h4> Zadanie 3 </h4>

Ostatnim zadaniem jest implementacji komórki i sieci LSTM. 

![lstm](https://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-chain.png)

* W klasie `LSTMCell` ma znaleźć się główna logika LSTMa, czyli wszystkie wagi do stanów `hidden` i `cell` jak i bramek kontrolujących te stany. 
* W klasie `LSTM` powinno znaleźć się wywołanie komórki LSTM, HINT: poprzednio było w pętli uczenia, teraz przenisiemy to do klasy modelu.
* W pętli uczenia należy uzupełnić brakujące wywołania do uczenia i ewaluacji modelu.

Zdecydowanie polecam [materiały Chrisa Olaha](http://colah.github.io/posts/2015-08-Understanding-LSTMs/) do zarówno zrozumienia jak i ściągi do wzorów.

Zadaniem jest osiągnięcie wartości `f1_score` lepszej niż na sieci RNN, przy prawidłowej implementacji nie powinno być z tym problemów używając podanych hiperparametrów. Dozwolona jest oczywiście zmiana `random seed`.

#### Komórka LSTM

In [836]:
class LSTMCell(torch.nn.Module):
    def __init__(self, input_size: int, hidden_size: int):
        super(LSTMCell, self).__init__()
    
        self.input_size = input_size
        self.hidden_size = hidden_size

        # initialize LSTM weights 
        # NOTE: there are different approaches that are all correct 
        # (e.g. single matrix for all input opperations), you can pick
        # whichever you like for this task

        self.W_i = torch.nn.Parameter(torch.randn(input_size, hidden_size))
        self.U_i = torch.nn.Parameter(torch.randn(hidden_size, hidden_size))
        self.b_i = torch.nn.Parameter(torch.randn(hidden_size))
    
        self.W_f = torch.nn.Parameter(torch.randn(input_size, hidden_size))
        self.U_f = torch.nn.Parameter(torch.randn(hidden_size, hidden_size))
        self.b_f = torch.nn.Parameter(torch.randn(hidden_size))
    
        self.W_o = torch.nn.Parameter(torch.randn(input_size, hidden_size))
        self.U_o = torch.nn.Parameter(torch.randn(hidden_size, hidden_size))
        self.b_o = torch.nn.Parameter(torch.randn(hidden_size))
    def forward(self, 
                input: torch.tensor, 
                states: Tuple[torch.tensor, torch.tensor]) -> Tuple[torch.tensor, torch.tensor]:
        
        hidden, cell = states
        
        # Compute input, forget, and output gates
        # then compute new cell state and hidden state
        # see http://colah.github.io/posts/2015-08-Understanding-LSTMs/ 
        input_gate = sigmoid(input @ self.W_i + hidden @ self.U_i + self.b_i)
        forget_gate = sigmoid(input @ self.W_f + hidden @ self.U_f + self.b_f)
        output_gate = sigmoid(input @ self.W_o + hidden @ self.U_o + self.b_o)
        
        cell = forget_gate * cell + input_gate * tanh(input @ W_c + hidden @ U_c + b_c)
        
        hidden = output_gate * tanh(cell)
        
        return hidden, cell

### Klasa modelu LSTM

In [837]:
class LSTM(torch.nn.Module):

    def __init__(self, 
                 input_size: int, 
                 hidden_size: int):
        """
        :param input_size: int
            Dimensionality of the input vector
        :param hidden_size: int
            Dimensionality of the hidden space
        """
        
        super(LSTM, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size

        self.cell = nn.LSTMCell(input_size=input_size, hidden_size=hidden_size)
        
    def forward(self, 
                input: torch.tensor) -> Tuple[torch.tensor, torch.tensor]:
        """
        :param input: torch.tensor 
            Input tesnor for a single observation at timestep t
            shape [batch_size, input_size]
        Returns Tuple of two torch.tensors, both of shape [seq_len, batch_size, hidden_size]
        """
        
        batch_size = input.shape[0]
        
        hidden, cell = self.init_hidden_cell(batch_size)
        
        hiddens = []
        cells = []
        
        # this time we will process the whole sequence in the forward method
        # as oppose to the previous exercise, remember to loop over the timesteps
        
        time_steps = input.shape[1]
        
        
        #najlatwiej w pętli
        for t in range(time_steps):
            hidden, cell = self.cell(input[:, t, :], (hidden, cell))
            hiddens.append(hidden)
            cells.append(cell)
    
        return torch.stack(hiddens), torch.stack(cells)
    
    def init_hidden_cell(self, batch_size):
        """
        Returns initial value for the hidden and cell states
        """
        return (torch.zeros(batch_size, self.hidden_size, requires_grad=True), 
                torch.zeros(batch_size, self.hidden_size, requires_grad=True))

### Pętla uczenia

In [838]:
from itertools import chain

# torch.manual_seed(1337)

# build data loaders
train_loader = DataLoader(train_dataset, batch_size=1, sampler=sampler)
test_loader = DataLoader(test_dataset, batch_size=1)

# initialize the lstm with an additional cliassifier layer at the top
lstm = LSTM(input_size=len(all_letters), hidden_size=128)
clf = torch.nn.Linear(in_features=128, out_features=len(label_to_idx))

# initialize a optimizer
params = chain(lstm.parameters(), clf.parameters())
optimizer = torch.optim.Adam(params, lr=0.01) 

# we will train for only a single epoch 
epoch = 1

# main loop
for epoch in range(epoch):
    
    loss_buffer = []
    
    for i, (x, y) in enumerate(train_loader):   
        
        optimizer.zero_grad()
        # don't forget about the classifier!
        hidden, state = lstm(x)
        output = clf(hidden[-1, :, :])
        
        # calucate the loss
        loss = cross_entropy(output, y)
        loss.backward()
        optimizer.step()                                
        
        loss_buffer.append(loss.item())
        
        if i % 1000 == 1:
            print(f"Epoch: {epoch} Progress: {100 * i/len(train_loader):2.0f}% Loss: {np.mean(loss_buffer):.3f}")
            loss_buffer = []
# evaluate on the test set
with torch.no_grad():
    
    ps = []
    ys = []
    for i, (x, y) in enumerate(test_loader): 
        
        ys.append(y.numpy())
        
        hidden, state = lstm(x)
        output = clf(hidden[-1, :, :])

        pred = output.argmax(dim=1)
        ps.append(pred.cpu().numpy())
    
    ps = np.concatenate(ps, axis=0)
    ys = np.concatenate(ys, axis=0)
    f1 = f1_score(ys, ps, average='weighted')
    
    print(f"Final F1 score: {f1:.2f}")
    assert f1 > 0.18, "You should get over 0.18 f1 score, try changing some hiperparams!"

Epoch: 0 Progress:  0% Loss: 2.920
Epoch: 0 Progress:  6% Loss: 2.456
Epoch: 0 Progress: 11% Loss: 1.936
Epoch: 0 Progress: 17% Loss: 1.668
Epoch: 0 Progress: 22% Loss: 1.597
Epoch: 0 Progress: 28% Loss: 1.455
Epoch: 0 Progress: 33% Loss: 1.400
Epoch: 0 Progress: 39% Loss: 1.304
Epoch: 0 Progress: 44% Loss: 1.363
Epoch: 0 Progress: 50% Loss: 1.277
Epoch: 0 Progress: 55% Loss: 1.183
Epoch: 0 Progress: 61% Loss: 1.182
Epoch: 0 Progress: 66% Loss: 1.158
Epoch: 0 Progress: 72% Loss: 1.118
Epoch: 0 Progress: 77% Loss: 1.082
Epoch: 0 Progress: 83% Loss: 1.082
Epoch: 0 Progress: 89% Loss: 1.051
Epoch: 0 Progress: 94% Loss: 0.974
Epoch: 0 Progress: 100% Loss: 1.004
Final F1 score: 0.24


<h4> Zadanie 4. </h4>

Zaimplementuj analogiczną do funkcji `predict` z zadania 2 dla modelu `lstm+clf`.


In [839]:
def predict_lstm(name: str, lstm: torch.nn.Module, clf: torch.nn.Module):
    """Prints the name and model's top 3 predictions with scores"""
    a = line_to_tensor(name)
    a= a.unsqueeze(0)
    _,predictions= lstm(a)
    hidden = predictions.squeeze(1)
    class_probs = clf(hidden)
    _, top_3_indices = class_probs.topk(3, dim=1)
    top_3_predictions = top_3_indices.detach().cpu().numpy()
    top_3_probs = class_probs.detach().cpu().numpy()[:,top_3_indices]
    dict_yall = {}
    for i in top_3_predictions:
        for x in i:
            try:
                dict_yall[x] += 1
            except:
                dict_yall[x] = 1
    nowy = sorted(dict_yall, key=dict_yall.get, reverse=True)[:3]
    for i in range(3):
        print(f"\tPrediction {i+1}: {decode_nationality(nowy[i])}")


In [840]:
# test your lstm predictor
some_names = ["Satoshi", "Jackson", "Schmidhuber", "Hinton", "Kowalski"]
    
for name in some_names:
    print(name)
    predict_lstm(name, lstm, clf)

Satoshi
	Prediction 1: [3, 'Arabic']
	Prediction 2: [12, 'Japanese']
	Prediction 3: [5, 'Korean']
Jackson
	Prediction 1: [6, 'English']
	Prediction 2: [15, 'Chinese']
	Prediction 3: [1, 'Scottish']
Schmidhuber
	Prediction 1: [7, 'Polish']
	Prediction 2: [9, 'French']
	Prediction 3: [12, 'Japanese']
Hinton
	Prediction 1: [15, 'Chinese']
	Prediction 2: [5, 'Korean']
	Prediction 3: [9, 'French']
Kowalski
	Prediction 1: [7, 'Polish']
	Prediction 2: [13, 'Czech']
	Prediction 3: [15, 'Chinese']
